### Запуск Spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark.sql.functions import *
import pandas as pd
import datetime
import os



# spark = SparkSession.builder \
#     .appName("S3Example") \
#     .config("spark.jars.packages",
#         "org.apache.hadoop:hadoop-aws:3.3.4,"
#         "com.amazonaws:aws-java-sdk-bundle:1.12.262,"
#         "ru.yandex.clickhouse:clickhouse-jdbc:0.3.2,"
#         "org.postgresql:postgresql:42.5.0,"
#         "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
#     .getOrCreate()


# hadoop_conf = spark._jsc.hadoopConfiguration()
# hadoop_conf.set("fs.s3a.access.key", os.getenv("MINIO_ROOT_USER"))
# hadoop_conf.set("fs.s3a.secret.key", os.getenv("MINIO_ROOT_PASSWORD"))
# hadoop_conf.set("fs.s3a.endpoint", "http://localhost:9001")
# hadoop_conf.set("fs.s3a.connection.ssl.enabled", "false")
# hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
# hadoop_conf.set("fs.s3a.path.style.access", "true")
# hadoop_conf.set("fs.s3a.threads.keepalivetime", "60000")  # миллисекунды!
# spark = SparkSession \
#             .builder \
#             .config("spark.master", "local") \
#             .config('spark.executor.cores', '8')\
#             .config('spark.executor.memory', '2g')\
#             .config('spark.driver.memory', '2g')\
#             .config('spark.dynamicAllocation.minExecutors', '4')\
#             .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
#             .config("spark.jars.packages", "ru.yandex.clickhouse:clickhouse-jdbc:0.3.2") \
#             .appName("spark_halltape") \
#     .getOrCreate()



spark = SparkSession \
            .builder \
            .config("spark.master", "local") \
            .config('spark.executor.cores', '8')\
            .config('spark.executor.memory', '2g')\
            .config('spark.driver.memory', '2g')\
            .config('spark.dynamicAllocation.minExecutors', '4')\
             .config("spark.jars.packages",
        "org.apache.hadoop:hadoop-aws:3.4.0,"
        "com.amazonaws:aws-java-sdk-bundle:1.12.262,"
        "ru.yandex.clickhouse:clickhouse-jdbc:0.3.2,"
        "org.postgresql:postgresql:42.5.0,"
        "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0") \
            .appName("test") \
    .getOrCreate()

hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.access.key", os.environ["MINIO_ROOT_USER"])
hadoop_conf.set("fs.s3a.secret.key", os.environ["MINIO_ROOT_PASSWORD"])
hadoop_conf.set("fs.s3a.endpoint", "http://minio:9000")
hadoop_conf.set("fs.s3a.connection.ssl.enabled", "false")
hadoop_conf.set("fs.s3a.path.style.access", "true")
hadoop_conf.set("fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")
hadoop_conf.set("fs.s3a.connection.timeout", "60000")
hadoop_conf.set("fs.s3a.socket.timeout", "60000")
hadoop_conf.set("fs.s3a.connection.establish.timeout", "60000")
hadoop_conf.set("fs.s3a.threads.keepalivetime", "60000")
hadoop_conf.set("fs.s3a.multipart.purge.age", "999999")


print("Активные Spark сессии:", 'http://localhost:4040/jobs')

:: loading settings :: url = jar:file:/opt/conda/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.3.jar!/org/apache/ivy/core/settings/ivysettings.xml
Ivy Default Cache set to: /home/jovyan/.ivy2.5.2/cache
The jars for the packages stored in: /home/jovyan/.ivy2.5.2/jars
org.apache.hadoop#hadoop-aws added as a dependency
com.amazonaws#aws-java-sdk-bundle added as a dependency
ru.yandex.clickhouse#clickhouse-jdbc added as a dependency
org.postgresql#postgresql added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a708a74c-3ead-4489-a50b-41e9fb977c49;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.4.0 in central
	found software.amazon.awssdk#bundle;2.23.19 in central
	found org.wildfly.openssl#wildfly-openssl;1.1.3.Final in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found ru.yandex.clickhouse#clickhouse-jdbc;0.3.2 in central
	found com.clickhouse#clickh

Активные Spark сессии: http://localhost:4040/jobs


In [4]:
hadoop_conf = spark._jsc.hadoopConfiguration()
conf_iter = hadoop_conf.iterator()
while conf_iter.hasNext():
    entry = conf_iter.next()
    print(f"{entry.getKey()} = {entry.getValue()}")

yarn.log-aggregation.file-formats = TFile
yarn.nodemanager.amrmproxy.wait.uam-register.done = false
ipc.server.tcpnodelay = true
fs.s3a.select.output.csv.record.delimiter = \n
mapreduce.jobhistory.client.thread-count = 10
hadoop.security.group.mapping.ldap.posix.attr.uid.name = uidNumber
yarn.admin.acl = *
yarn.app.mapreduce.am.job.committer.cancel-timeout = 60000
yarn.federation.enabled = false
yarn.nodemanager.log.trigger.delete.by-size.enabled = false
mapreduce.job.emit-timeline-data = false
fs.s3a.select.input.csv.quote.character = "
yarn.nodemanager.runtime.linux.sandbox-mode.local-dirs.permissions = read
yarn.resourcemanager.leveldb-state-store.path = ${hadoop.tmp.dir}/yarn/system/rmstore
ipc.client.connection.maxidletime = 10000
yarn.federation.cache.class = org.apache.hadoop.yarn.server.federation.cache.FederationJCache
yarn.nodemanager.health-checker.scripts = script
yarn.nodemanager.process-kill-wait.ms = 5000
yarn.minicluster.use-rpc = false
io.map.index.interval = 128
mapre

In [3]:
# Создадим dataframe словарь
# Путь к файлу в S3
s3_path = "s3a://prod/api/earthquake/events_2025-07-29.json"
# s3_path_regions = "s3a://prod/jdbc/regions/*.parquet"

# Чтение JSON-файла
df = spark.read.json(s3_path)
# regions = spark.read.parquet(s3_path_regions)

# # Покажем несколько строк
# df.show(truncate=False)

flattened = df.selectExpr("explode(features) as feature") \
    .select(
        col("feature.id").alias("id"),
        from_unixtime(col("feature.properties.time") / 1000).alias("time"),
        col("time").cast("date").alias("load_date"),
         #trim(split(col("feature.properties.place"), ",").getItem(0)).alias("place"),
        # trim(split(col("feature.properties.place"), ",").getItem(1)).alias("initial_region"), 
        col("feature.properties.mag").alias("magnitude"),
        col("feature.properties.felt").alias("felt"),
        col("feature.properties.tsunami").alias("tsunami"),
        col("feature.properties.url").alias("url"),
        col("feature.geometry.coordinates")[0].alias("longitude"),
        col("feature.geometry.coordinates")[1].alias("latitude"),
        col("feature.geometry.coordinates")[2].alias("depth"),
        md5(lower(trim(col("feature.properties.place")))).alias("place_hash")
    )
flattened.show()

# flattened.alias("f")\
#     .join(regions.alias("r"), "place_hash", "left")\
#     .where("initial_region IS NULL") \
#     .drop('initial_region', 'place_hash') \
#     .show(truncate=False)

# felt - Сколько людей сообщили, что ощутили землетрясение (в данном случае 1 человек)
# tsunami - Флаг цунами: 1 — возможно, 0 — нет
# url - Ссылка на страницу события на сайте USGS


+------------+-------------------+----------+---------+----+-------+--------------------+-----------------+----------------+-------+--------------------+
|          id|               time| load_date|magnitude|felt|tsunami|                 url|        longitude|        latitude|  depth|          place_hash|
+------------+-------------------+----------+---------+----+-------+--------------------+-----------------+----------------+-------+--------------------+
|tx2025ovmkxi|2025-07-29 23:57:55|2025-07-29|      1.5|NULL|      0|https://earthquak...|         -104.266|          31.648| 5.7922|645cdf3b96665cbdc...|
|  us6000qw6i|2025-07-29 23:56:30|2025-07-29|      5.2|NULL|      0|https://earthquak...|         157.6579|        -57.8747|   10.0|9b1686c8417d94769...|
|  us6000qw6m|2025-07-29 23:48:52|2025-07-29|      5.4|NULL|      0|https://earthquak...|         159.5076|         51.6482|   10.0|ffbd2533908d37990...|
|  us6000qw6g|2025-07-29 23:48:16|2025-07-29|      5.6|NULL|      0|https://

In [6]:
flattened.show(3)

+------------+-------------------+----------+---------+----+-------+--------------------+---------+--------+------+--------------------+
|          id|               time| load_date|magnitude|felt|tsunami|                 url|longitude|latitude| depth|          place_hash|
+------------+-------------------+----------+---------+----+-------+--------------------+---------+--------+------+--------------------+
|tx2025ovmkxi|2025-07-29 23:57:55|2025-07-29|      1.5|NULL|      0|https://earthquak...| -104.266|  31.648|5.7922|645cdf3b96665cbdc...|
|  us6000qw6i|2025-07-29 23:56:30|2025-07-29|      5.2|NULL|      0|https://earthquak...| 157.6579|-57.8747|  10.0|9b1686c8417d94769...|
|  us6000qw6m|2025-07-29 23:48:52|2025-07-29|      5.4|NULL|      0|https://earthquak...| 159.5076| 51.6482|  10.0|ffbd2533908d37990...|
+------------+-------------------+----------+---------+----+-------+--------------------+---------+--------+------+--------------------+
only showing top 3 rows


In [30]:
print(spark.sparkContext._jvm.org.apache.hadoop.util.VersionInfo.getVersion())

3.4.1


In [ ]:
ch_max_df = spark.read \
    .format("jdbc") \
    .option("url", 'jdbc:clickhouse://clickhouse:8123/default',) \
    .option("user", "admin") \
    .option("password", "admin") \
    .option("dbtable", f"(SELECT MAX(load_date) as max_date FROM enriched_earthquakes)") \
    .option("driver", "com.clickhouse.jdbc.ClickHouseDriver") \
    .load()

max_date = ch_max_df.collect()[0]["max_date"].strftime("%Y-%m-%d")

if max_date == '1970-01-01':
    predicate = "1=1"
else:
    predicate = f"load_date > '{max_date}'"

predicate

In [ ]:
spark.stop()

### Сборка витрины данных

In [ ]:
PATH = '/home/jovyan/source'

def debit_card(date):
    card = spark.read.csv(f"{PATH}/card.csv", header=True, sep=";")\
                .where(f''' load_date = "{date}" ''')
    
    status_card = spark.read.csv(f"{PATH}/Status_card.csv", header=True, sep=";")\
                        .where(f''' load_date = "{date}"  ''')
    
    transactions = spark.read.csv(f"{PATH}/transactions.csv", header=True, sep=";")\
                        .where(f''' load_date = "{date}"  ''')
    
    
    first_trx = transactions.groupBy('card_num')\
                            .agg(F.min('transaction_datetime').alias('transaction_datetime'))
    
    
    first_trx_info = first_trx.join(transactions, ['card_num','transaction_datetime'], 'inner')
    
    dt_trx = first_trx_info.select('card_num', 'amount', '*')
    df_st = status_card.select('card_num', 'card_num_md5', 'status')
    
    result_df = dt_trx\
                    .join(df_st, "card_num", "inner")\
                    .join(card, "card_num_md5", "right")\
                    .drop('card_num_md5')
    
    final_df = result_df\
                    .where(''' card_num IS NOT NULL ''')\
                    .groupBy('card_num',
                             'transaction_datetime',
                             'status',
                             'card_order_dt',
                             'url',
                             'cookie')\
                    .agg(F.max('amount').alias('amt'))
    
    
    datamart = final_df\
                    .select('card_order_dt',
                            'card_num',
                            'cookie',
                            'url',
                            'amt',
                            'status')\
                    .withColumn('transaction_level',
                                    F.when(F.col('amt') > 300, True).otherwise(False))\
                    .withColumn('status_flag',
                                    F.when(F.col('status') == "выдана", True).otherwise(False))\
                    .withColumn('partition_date', F.lit(date).cast('string').alias('partition_date'))\
                    .withColumn('load_date', F.lit(date).cast('string').alias('load_date'))\
                    .drop('amt', 'status')
    
    datamart.write.mode("append").partitionBy('partition_date').csv('data_lake/debit_card', header=True)



start = '2024-07-01'
end = '2024-07-10'

for dt in pd.date_range(start, end):
    date = str(dt)[:10]
    print(date)
    debit_card(date)